In [1]:
# Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Model
import lightgbm as lgb
#import shap
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

# Configuration
import warnings
warnings.filterwarnings('ignore')

D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
df = pd.read_csv('train.csv', parse_dates=['date'],index_col=['date'])

In [5]:
df

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10
...,...,...,...
2017-12-27,10,50,63
2017-12-28,10,50,59
2017-12-29,10,50,74


In [6]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofweek'] = df.index.dayofweek
df['weekofyear']=df.index.weekofyear
df['is_weekend']=df.index.dayofweek

In [7]:
df

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2,52,2
2017-12-28,10,50,59,28,12,2017,3,52,3
2017-12-29,10,50,74,29,12,2017,4,52,4


In [8]:
df_test = pd.read_csv('C:/Users/dev/Downloads/test.csv',parse_dates=['date'],index_col=['date'])
df_test['day'] = df_test.index.day
df_test['month'] = df_test.index.month
df_test['year'] = df_test.index.year
df_test['dayofweek'] = df_test.index.dayofweek
df_test['weekofyear']  = df_test.index.weekofyear
df_test['is_weekend'] = df_test.index.dayofweek 
df_test

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/dev/Downloads/test.csv'

In [9]:
x_train = df.loc[df['year'] < 2017].drop(['sales'], axis=1)
y_train = df.loc[df['year'] < 2017, 'sales']

x_test = df.loc[df['year'] >= 2017].drop(['sales'], axis=1).reset_index(drop=True)
y_test = df.loc[df['year'] >= 2017, 'sales'].reset_index(drop=True)

In [10]:
first_model = lgb.LGBMRegressor(random_state=384).fit(x_train, y_train, 
                                                      eval_metric= lambda y_true, y_pred: [lgbm_smape(y_true, y_pred)])

In [11]:
#SMAPE: Symmetric mean absolute percentage error (adjusted MAPE)
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

def lgbm_smape(y_true, y_pred):
    smape_val = smape(y_true, y_pred)
    return 'SMAPE', smape_val, False
print("TRAIN SMAPE:", smape(y_train, first_model.predict(x_train)))
print("TEST SMAPE:", smape(y_test, first_model.predict(x_test)))

TRAIN SMAPE: 14.609660713139665
TEST SMAPE: 13.846214754280195


In [12]:
x_test

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2017,6,52,6
1,1,1,2,1,2017,0,1,0
2,1,1,3,1,2017,1,1,1
3,1,1,4,1,2017,2,1,2
4,1,1,5,1,2017,3,1,3
...,...,...,...,...,...,...,...,...
182495,10,50,27,12,2017,2,52,2
182496,10,50,28,12,2017,3,52,3
182497,10,50,29,12,2017,4,52,4
182498,10,50,30,12,2017,5,52,5


In [13]:
preds = first_model.predict(x_test)

In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
 mean_absolute_percentage_error(y_test,preds)

14.145734110836939

In [18]:
from sklearn.metrics import mean_squared_error

ms = mean_squared_error(y_test, preds, squared = False)
ms

9.689344430087539

In [17]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,preds)
r2

0.9056985908414675

In [24]:
# Add weights to most recent data
train_main = pd.concat([df,
                        df.loc[df.index > df.index.max()- pd.DateOffset(30, 'D')],
                        df.loc[df.index > df.index.max()- pd.DateOffset(30, 'D')],
                        df.loc[df.index > df.index.max()- pd.DateOffset(15, 'D')]
                       ])
# Train a model
first_model.fit(X=train_main[['store', 'item','year','month','dayofweek','weekofyear','is_weekend']], y=train_main['sales'])

# Get predictions for the test set
df_test['sales'] = first_model.predict(df_test[['store', 'item','year','month','dayofweek','weekofyear','is_weekend']])

In [25]:
df_test['sales']

date
2018-01-01    12.078175
2018-01-02    14.050705
2018-01-03    14.050705
2018-01-04    13.957468
2018-01-05    16.299989
                ...    
2018-03-27    68.373865
2018-03-28    68.373865
2018-03-29    73.117121
2018-03-30    75.648606
2018-03-31    79.432924
Name: sales, Length: 45000, dtype: float64